In [40]:
from Compysite import Material, Lamina, Laminate
import numpy as np
import matplotlib.pyplot as plt
# from conversion import create_tensor_3D

from conversion import (
    to_epsilon,
    to_gamma,
    tensor_to_vec,
    T_z,
    transformation_3D,
    ConversionMatrices,
)

### Validation 1

In [67]:
E_f = np.array([233, 23.1, 23.1])
v_f = np.array([0.40, 0.20, 0.20])
G_f = np.array([8.27, 8.96, 8.96])
alpha_f = np.array([-0.54, 10.10, 10.10])
mat_f = Material(E_f, v_f, G_f, alpha_f)
V_f = 0.61

E_m = 4.62
v_m = 0.36
G_m = 0
alpha_m = 41.4
mat_m = Material(E_m, v_m, G_m, alpha_m)

layer_1 = Lamina(mat_fiber=mat_f, mat_matrix=mat_m, Vol_fiber=V_f, array_geometry=2)

E, v, G = layer_1.get_lamina_properties()
a, b = layer_1.get_lamina_expansion_properties()

print('E',E)
print('v',v)
print('G',G)
print('alpha',a)

E [143.9318      12.03631579  12.03631579]
v [0.58919839 0.2624     0.2624    ]
G [3.78691416 4.54567129 4.54567129]
alpha [-1.49770933e-02  2.81195600e+01  2.81195600e+01]


### Validation 2

In [70]:
E = np.array([19.2, 1.56, 1.56])
v = np.array([0.59, 0.24, 0.24])
G = np.array([0.49, 0.82, 0.82])

lam = Laminate()
mat = Material(E, v, G)
layer_1 = Lamina(mat_composite=mat)

print('S')
print(layer_1.matrices.S)
print()
print('C')
print(layer_1.matrices.C)

S
[[ 0.05208333 -0.0125     -0.0125      0.          0.          0.        ]
 [-0.0125      0.64102564 -0.37820513  0.          0.          0.        ]
 [-0.0125     -0.37820513  0.64102564  0.          0.          0.        ]
 [ 0.          0.          0.          2.04081633  0.          0.        ]
 [ 0.          0.          0.          0.          1.2195122   0.        ]
 [ 0.          0.          0.          0.          0.          1.2195122 ]]

C
[[19.6485623   0.93450479  0.93450479  0.          0.          0.        ]
 [ 0.93450479  2.43745102  1.45631895  0.          0.          0.        ]
 [ 0.93450479  1.45631895  2.43745102  0.          0.          0.        ]
 [ 0.          0.          0.          0.49        0.          0.        ]
 [ 0.          0.          0.          0.          0.82        0.        ]
 [ 0.          0.          0.          0.          0.          0.82      ]]


### Validation 3

In [4]:
E = np.array([163, 14.1, 14.1]) * 1e9
v = np.array([0.45, 0.24, 0.24])
G = np.array([3.6, 4.8, 4.8]) * 1e9
alpha = np.array([-0.018, 24.3, 24.3, 0, 0, 0]) * 1e-6
beta = np.array([150, 4870, 4870, 0, 0, 0]) * 1e-6

sigma = create_tensor_3D(50, -50, -5, 0, 0, -3) * 1e6

lam = Laminate()
mat = Material(E, v, G)
layer_1 = Lamina(mat_composite=mat)
lam.add_lamina(layer_1)

layer_1.apply_stress(sigma)
e_thermal = alpha * 10
e_moisture = beta * 0.6

print((layer_1.local_state.strain + e_thermal + e_moisture))

[ 0.00047755 -0.00029514  0.00433252  0.          0.         -0.000625  ]


### Validation 4

In [5]:
E = np.array([163, 14.1, 14.1]) * 1e9
v = np.array([0.45, 0.24, 0.24])
G = np.array([3.6, 4.8, 4.8]) * 1e9
alpha = np.array([-0.018, 24.3, 24.3, 0, 0, 0]) * 1e-6
beta = np.array([150, 4870, 4870, 0, 0, 0]) * 1e-6

lam = Laminate()
mat = Material(E, v, G)
layer_1 = Lamina(mat_composite=mat)
lam.add_lamina(layer_1)

epsilon = np.array([4.0e-4, -3.5e-3, 1.2e-3, 0, 0, -6e-4])
e_thermal = alpha * -30
e_moisture = beta * 0.6
e_total = create_tensor_3D(*(epsilon - e_thermal - e_moisture))

layer_1.apply_strain(e_total)
print(layer_1.local_state.stress * 1e-6)

[   9.47654588 -108.19637856  -62.49293028    0.            0.
   -2.88      ]


### Validation 5

In [6]:
E = np.array([163, 14.1, 14.1]) * 1e9
v = np.array([0.45, 0.24, 0.24])
G = np.array([3.6, 4.8, 4.8]) * 1e9
alpha = np.array([-0.018, 24.3, 24.3, 0, 0, 0]) * 1e-6
beta = np.array([150, 4870, 4870, 0, 0, 0]) * 1e-6

lam = Laminate()
mat = Material(E, v, G)
layer_1 = Lamina(mat_composite=mat)
lam.add_lamina(layer_1)

epsilon = create_tensor_3D(4.0e-4, -3.5e-3, 1.2e-3, 0, 0, -6e-4)

layer_1.apply_strain(epsilon)

print(layer_1.matrices.C_reduced.dot(np.array([4.0e-4, -3.5e-3, -6e-4])) * 1e-6)
print(layer_1.local_state.stress * 1e-6)

[ 53.62318161 -48.23674327  -2.88      ]
[ 51.99071907 -50.3710594   -4.66761113   0.           0.
  -2.88      ]


### Validation 6

In [72]:
E = np.array([100, 20, 20])
v = np.array([0.40, 0.18, 0.18])
G = np.array([4, 5, 5])

lam = Laminate()
mat = Material(E, v, G)
layer_1 = Lamina(mat_composite=mat)
layer_2 = Lamina(mat_composite=mat)

lam.add_lamina(layer_1, 45)
lam.add_lamina(layer_2, -30)

print('--- 45 degree ---')
print(lam.get_lamina(1).matrices.T_2D)
print(lam.get_lamina(1).matrices.S_bar_reduced)
print(lam.get_lamina(1).matrices.Q_bar_reduced)
print('--- -30 degree ---')
print(lam.get_lamina(2).matrices.T_2D)
print(lam.get_lamina(2).matrices.S_bar_reduced)
print(lam.get_lamina(2).matrices.Q_bar_reduced)

--- 45 degree ---
[[ 0.5  0.5  1. ]
 [ 0.5  0.5 -1. ]
 [-0.5  0.5  0. ]]
[[ 0.0641 -0.0359 -0.02  ]
 [-0.0359  0.0641 -0.02  ]
 [-0.02   -0.02    0.0636]]
[[37.007408   27.007408   20.13044529]
 [27.007408   37.007408   20.13044529]
 [20.13044529 20.13044529 28.38392785]]
--- -30 degree ---
[[ 0.75       0.25      -0.8660254]
 [ 0.25       0.75       0.8660254]
 [ 0.4330127 -0.4330127  0.5      ]]
[[ 0.045575   -0.027375    0.04685197]
 [-0.027375    0.065575   -0.01221096]
 [ 0.04685197 -0.01221096  0.0977    ]]
[[ 62.98383525  21.16142604 -27.55901479]
 [ 21.16142604  22.72294468  -7.30793923]
 [-27.55901479  -7.30793923  22.53794589]]


### For 2D Material Properties:

$E_1=170 GPa, E_2=9 GPa, \nu_{12}=0.27, G_{12}=4.4 GPa$

In [16]:
E = np.array([170, 9, 1])
v = np.array([0, 0.27, 0.27])
G = np.array([1, 4.4, 4.4])

### SEE VALIDATION 5 FOR HOW TO GET 2D STRESS STATE

### EXAM 1
#### MCE 440 - Composites
#### March 29, 2022

#### Question 1


In [62]:
E = np.array([180, 20, 20])*1e9
v = np.array([0, 0.3, 0.3])
G = np.array([1, 5, 5])*1e9

lam = Laminate()
mat = Material(E, v, G)
layer_1 = Lamina(mat_composite=mat)
lam.add_lamina(layer_1, 60)

sigma = create_tensor_3D(50, 10, -10)
lam.apply_stress(sigma)

print(lam.lamina[0].matrices.S_bar_reduced*1e9)

[[ 0.06534722 -0.028125    0.01130644]
 [-0.028125    0.043125   -0.04979646]
 [ 0.01130644 -0.04979646  0.09416667]]



In [21]:
print(lam.lamina[0].matrices.Q_bar_reduced)

[[ 28.75        37.91666667  16.59882024]
 [ 37.91666667 109.55808081  53.38303057]
 [ 16.59882024  53.38303057  36.85606061]]


In [74]:
a = lam.lamina[0].matrices.S_bar_reduced.dot(np.array([50, 10, -10]))
a

array([ 3.046, -0.954, -1.836])

In [39]:
lam.global_state[0].strain

array([ 2.99027778e-09, -9.62500000e-10, -5.33333333e-10,  0.00000000e+00,
        0.00000000e+00,  8.17912881e-11])

In [53]:
lam.lamina[0].local_state.stress

array([ 20.        ,  40.        , -10.        ,   0.        ,
         0.        , -17.32050808])